# Data Preprocessing

## 1. Upscaling images

In [1]:
import cv2 as cv
import numpy as np
from os import listdir, walk                  
from os.path import isfile, join

from pathlib import Path

from PIL import Image

import torch
import torchvision
import torchvision.transforms as T

from super_image import DrlnModel, ImageLoader
import requests

In [2]:


def walk_through_dir(dir_path: Path) -> dict:
    """Prints dir_path content"""
    counts = dict()
    classes = dict()
    i=-1
    for dirpath, dirnames, filenames in walk(dir_path):
        i = i+1
        print(f"There are {len(dirnames)} directiories and {len(filenames)} images in '{dirpath}' folder ")
        if (i>0):
            c = dirpath.split('\\')
            counts[c[-1]] = len(filenames)
            classes[c[-1]] = i
    return (counts, classes)


data_path = Path("Kaggle/CV/Emotions_detection")
images_path = data_path / "emotions_dataset"
train_path = images_path / "train"
test_path = images_path / "test"
ctrain, classes = walk_through_dir(train_path)
ctest, _ = walk_through_dir(test_path)

There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train' folder 
There are 0 directiories and 3995 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\angry' folder 
There are 0 directiories and 436 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\disgusted' folder 
There are 0 directiories and 4097 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\fearful' folder 
There are 0 directiories and 7215 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\happy' folder 
There are 0 directiories and 4965 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\neutral' folder 
There are 0 directiories and 4830 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\sad' folder 
There are 0 directiories and 3171 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\surprised' folder 
There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\test' folder 
Th

In [3]:
image = Image.open(f"Kaggle/CV/Emotions_detection/emotions_dataset/train/happy/im10.png")

model = DrlnModel.from_pretrained('eugenesiow/drln', scale=4)
inputs = ImageLoader.load_image(image)
preds = model(inputs)
preds = model(preds)

ImageLoader.save_image(preds, './scaled_16x.png')
ImageLoader.save_compare(inputs, preds, './scaled_16x_compare.png')

c:\Users\Nevena\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
c:\Users\Nevena\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:680: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
c:\Users\Nevena\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'url_to_filename' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` to benefit from the new cache layout.
  warnings.warn(warning_message, FutureWarning)


https://huggingface.co/eugenesiow/drln/resolve/main/pytorch_model_4x.pt


c:\Users\Nevena\AppData\Local\Programs\Python\Python311\Lib\site-packages\super_image\data\loader.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.as_tensor([lr])


## Upscaling function

In [33]:
model = DrlnModel.from_pretrained('eugenesiow/drln', scale=4)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)

def upscale_4(image):
    preds = model(image.to(device))
    return preds

def upscale(path, multiple, data_type, data_class, index):
    print(path)
    
    image = Image.open(path)
    # image = torch.from_numpy(image).to(device)

    inputs = ImageLoader.load_image(image)
    # inputs.to(device)
    for i in range(0, multiple):
        inputs = upscale_4(inputs)#.detach().cpu().numpy()
    savepath = f"upscaled_images/{data_type}/{data_class}/im{index}_{4**multiple}.png"
    #torch.from_numpy(inputs).to(torch.device("cpu"))
    ImageLoader.save_image(inputs, savepath)

https://huggingface.co/eugenesiow/drln/resolve/main/pytorch_model_4x.pt
cuda:0


In [19]:
def upscale_all_images(multiple):
    c_train, classes = walk_through_dir(train_path)
    c_test, _ = walk_through_dir(test_path)

    for label, count in c_train.items():
        for i in range(0, count):
            path = f"Kaggle/CV/Emotions_detection/emotions_dataset/train/{label}/im{i}.png"
            upscale(path, multiple, "train", label, i)

    for label, count in c_test.items():
        for i in range(0, count):
            path = f"Kaggle/CV/Emotions_detection/emotions_dataset/test/{label}/im{i}.png"
            upscale(path, multiple, "test", label, i)
            
    

In [34]:
upscale_all_images(1)

There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train' folder 
There are 0 directiories and 3995 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\angry' folder 
There are 0 directiories and 436 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\disgusted' folder 
There are 0 directiories and 4097 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\fearful' folder 
There are 0 directiories and 7215 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\happy' folder 
There are 0 directiories and 4965 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\neutral' folder 
There are 0 directiories and 4830 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\sad' folder 
There are 0 directiories and 3171 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\train\surprised' folder 
There are 7 directiories and 0 images in 'Kaggle\CV\Emotions_detection\emotions_dataset\test' folder 
Th